# Medical Notes Classification

Medical notes is an useful information source for patient data extraction. Notes classification is also an important task in Medical NLP domain. There are many techniques to solve this problem ranging from traditional method (Logistic Regression, SVM,...) to the state-of-the-art models (Transformer).

The below code block is the baseline model for a text classification problem in medical domain.

* Input: the corpus of medical transcriptions.
* Output: the type of each notes.

In this problem, we try to classify five labels:
* Surgery
* Consult - History and Phy.
* Cardiovascular / Pulmonary
* Orthopedic
* Others

The train-test split was also defined, please don't change our split.

Metric to evaluate: `f1_macro`

# Baseline Model Result


0.3729330560342061

                                precision    recall  f1-score   support

    Cardiovascular / Pulmonary       0.35      0.39      0.37       148
    Consult - History and Phy.       0.32      0.06      0.10       207
                    Orthopedic       0.39      0.14      0.21       142
                         Other       0.66      0.74      0.70      1055
                       Surgery       0.43      0.57      0.49       435

                      accuracy                           0.56      1987
                     macro avg       0.43      0.38      0.37      1987
                  weighted avg       0.54      0.56      0.53      1987

# Library & Data Loading

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from simpletransformers.classification import ClassificationModel, ClassificationArgs

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


### PLEASE DON'T CHANGE ANYTHING IN THIS SECTION ###
DATA = "https://github.com/socd06/private_nlp/raw/master/data/mtsamples.csv"

filtered_labels = [
    "Surgery",
    "Consult - History and Phy.",
    "Cardiovascular / Pulmonary",
    "Orthopedic",
]
data = pd.read_csv(DATA, usecols=['medical_specialty', 'transcription']).dropna()
data.columns = ['labels', 'text']
data['labels'] = [i.strip() if (i.strip() in filtered_labels) else 'Other' for i in data.labels.to_list()]
train, test = train_test_split(data, test_size=0.4, stratify=data.labels, random_state=0)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
### END ###

# Preprocessing

# My Model


In [2]:
#encode labels 
le = LabelEncoder()
train['labels'] = le.fit_transform(train.labels)
test['labels'] = le.transform(test.labels)

### Encode Text


In [3]:
#create model
import TopicAllocate as ta
model = ta.Topic_Allocate()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haimi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haimi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#encode texts into matrix
X_train = np.asarray(model.doc2vec(train['text'], vector_size = 500, fit = True))
X_test = np.asarray(model.doc2vec(test['text'], vector_size = 500))


C:\Users\haimi\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\haimi\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:162: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
C:\Users\haimi\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [5]:
# onehot labels
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

y_train = train['labels']

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
Y_train = onehot_encoder.fit_transform(np.array(y_train).reshape(-1, 1))
Y_test = onehot_encoder.transform(np.array(test['labels']).reshape(-1,1))

# invert first example
inverted = le.inverse_transform([np.argmax(Y_train[0, :])])
print(inverted)

['Consult - History and Phy.']


### Train with LSTM

#### sequence -> vec

In [6]:
#%% import library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [7]:
max_size = np.amax(np.array([x.shape[0] for x in X_train]))
def fill_zeros(x, vector_size):
    try:
        missing = max_size - x.shape[0]
        fill_in = np.zeros((missing, vector_size))
        return np.vstack((fill_in, x))
    except:
        return np.zeros((max_size, vector_size))
func = lambda x: fill_zeros(x, 500)
X_train_lstm_s2v = np.array([func(x) for x in X_train])
X_test_lstm_s2v = np.array([func(x) for x in X_test])


In [8]:
#%% Xay dung model LSTM
lstm_model = Sequential()
# Units: ???
# return_sequences: tra lai ket qua cuoi cho lop tiep theo
lstm_model.add(LSTM(units=500, return_sequences=True, input_shape=(X_train_lstm_s2v.shape[1], X_train_lstm_s2v.shape[2])))
# Dropout: bỏ 20% chống overhitting
lstm_model.add(Dropout(0.2))
# regressor.add(LSTM(units=50, return_sequences=True))
# regressor.add(Dropout(0.2))
# regressor.add(LSTM(units=50, return_sequences=True))
# regressor.add(Dropout(0.2))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=5, activation="softmax"))
lstm_model.compile(optimizer="adam", loss="categorical_crossentropy",  metrics=["accuracy"])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 74, 50)            110200    
_________________________________________________________________
dropout (Dropout)            (None, 74, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 255       
Total params: 130,655
Trainable params: 130,655
Non-trainable params: 0
_________________________________________________________________


In [10]:
lstm_model.fit(X_train_lstm_s2v, Y_train, epochs=10, batch_size=32, validation_split= 0.1)

Epoch 1/10
84/84 [==============================] - 7s 81ms/step - loss: nan - accuracy: 0.0746 - val_loss: nan - val_accuracy: 0.0772
Epoch 2/10
84/84 [==============================] - 6s 72ms/step - loss: nan - accuracy: 0.0746 - val_loss: nan - val_accuracy: 0.0772
Epoch 3/10
84/84 [==============================] - 5s 58ms/step - loss: nan - accuracy: 0.0746 - val_loss: nan - val_accuracy: 0.0772
Epoch 4/10
84/84 [==============================] - 5s 54ms/step - loss: nan - accuracy: 0.0746 - val_loss: nan - val_accuracy: 0.0772
Epoch 5/10
84/84 [==============================] - 4s 51ms/step - loss: nan - accuracy: 0.0746 - val_loss: nan - val_accuracy: 0.0772
Epoch 6/10
84/84 [==============================] - 5s 54ms/step - loss: nan - accuracy: 0.0746 - val_loss: nan - val_accuracy: 0.0772
Epoch 7/10
84/84 [==============================] - 5s 54ms/step - loss: nan - accuracy: 0.0746 - val_loss: nan - val_accuracy: 0.0772
Epoch 8/10
65/84 [======================>.......] - ETA

In [ ]:
from sklearn.metrics import confusion_matrix
check = lstm_model.predict(X_test_lstm_s2v)
check = np.argmax(check, axis = 1)
ytrain = np.argmax(Y_test, axis = 1)
print(confusion_matrix(check, ytrain))
print(metrics.f1_score(ytrain, check, average='macro'))
print(metrics.classification_report(ytrain, check, target_names=list(le.classes_)))

2021-10-06 00:40:38.260557: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 723268000 exceeds 10% of free system memory.


[[ 33  29   0  50  30]
 [  7  27  12  96   0]
 [  1   8  35  40  49]
 [ 52 143  57 712 199]
 [ 55   0  38 157 157]]
0.3303601816063591
                            precision    recall  f1-score   support

Cardiovascular / Pulmonary       0.23      0.22      0.23       148
Consult - History and Phy.       0.19      0.13      0.15       207
                Orthopedic       0.26      0.25      0.25       142
                     Other       0.61      0.67      0.64      1055
                   Surgery       0.39      0.36      0.37       435

                  accuracy                           0.49      1987
                 macro avg       0.34      0.33      0.33      1987
              weighted avg       0.47      0.49      0.47      1987



In [ ]:
texts = train['text']
texts = [text.split() for text in texts]
lent  = np.array([len(text) for text in texts])
x = np.argmin(lent)
print(train['text'][x])

SUBJECTIVE:,
